This notebook is a sample code with Japanese comments.

# ハイパーパラメータの調整

In [5]:
import numpy as np
import pandas as pd


train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')

data = pd.concat([train, test], sort=False)

data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['Age'].fillna(data['Age'].median(), inplace=True)
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

In [6]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone
0,1,0.0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0,2,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1,2,0
2,3,1.0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,1,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0,2,0
4,5,0.0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0,1,1


In [7]:
delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)

In [8]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,3,0,22.0,1,0,7.2500,0,2,0
1,1,1,38.0,1,0,71.2833,1,2,0
2,3,1,26.0,0,0,7.9250,0,1,1
3,1,1,35.0,1,0,53.1000,0,2,0
4,3,0,35.0,0,0,8.0500,0,1,1


## LightGBM

In [9]:
from sklearn.model_selection import train_test_split


X_train, X_valid, y_train, y_valid = \
    train_test_split(X_train, y_train, test_size=0.3,
                                 random_state=0, stratify=y_train)

In [10]:
categorical_features = ['Embarked', 'Pclass', 'Sex']

In [16]:
import lightgbm as lgb


lgb_train = lgb.Dataset(X_train, y_train,
                                         categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train,
                                         categorical_feature=categorical_features)

model = lgb.train(params, lgb_train,
                               valid_sets=[lgb_train, lgb_eval],
                               verbose_eval=10,
                               num_boost_round=1000,
                               early_stopping_rounds=10)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.505699	valid_1's binary_logloss: 0.532106
[20]	training's binary_logloss: 0.427825	valid_1's binary_logloss: 0.482279
[30]	training's binary_logloss: 0.377242	valid_1's binary_logloss: 0.456641
[40]	training's binary_logloss: 0.345424	valid_1's binary_logloss: 0.447083
[50]	training's binary_logloss: 0.323113	valid_1's binary_logloss: 0.440407
[60]	training's binary_logloss: 0.302727	valid_1's binary_logloss: 0.434527
[70]	training's binary_logloss: 0.285597	valid_1's binary_logloss: 0.434932
Early stopping, best iteration is:
[66]	training's binary_logloss: 0.293072	valid_1's binary_logloss: 0.433251


In [17]:
y_pred[:10]

array([0.03605598, 0.40306884, 0.10732166, 0.0802399 , 0.46011271,
       0.20222002, 0.64929492, 0.11896033, 0.7452973 , 0.01917651])

# Optunaを使う

In [20]:
import optuna
from sklearn.metrics import log_loss

# max_bin  & num_leaves
def objective(trial):
    params = {
        'objective': 'binary',
        'max_bin': trial.suggest_int('max_bin', 255, 500),
        'learning_rate': 0.05,
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }

    lgb_train = lgb.Dataset(X_train, y_train,
                                             categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train,
                                            categorical_feature=categorical_features)

    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    score = log_loss(y_valid, y_pred_valid)
    return score

In [21]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)

/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning:

Using categorical_feature in Dataset.



Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.505699	valid_1's binary_logloss: 0.532106
[20]	training's binary_logloss: 0.427825	valid_1's binary_logloss: 0.482279
[30]	training's binary_logloss: 0.377242	valid_1's binary_logloss: 0.456641
[40]	training's binary_logloss: 0.345424	valid_1's binary_logloss: 0.447083
[50]	training's binary_logloss: 0.323113	valid_1's binary_logloss: 0.440407
[60]	training's binary_logloss: 0.302727	valid_1's binary_logloss: 0.434527
[70]	training's binary_logloss: 0.285597	valid_1's binary_logloss: 0.434932
Early stopping, best iteration is:
[66]	training's binary_logloss: 0.293072	valid_1's binary_logloss: 0.433251


[I 2020-04-30 02:31:39,080] Finished trial#0 resulted in value: 0.4332512137886331. Current best value is 0.4332512137886331 with parameters: {'max_bin': 427, 'num_leaves': 79}.
/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning:

Using categorical_feature in Dataset.



Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.505699	valid_1's binary_logloss: 0.532106
[20]	training's binary_logloss: 0.427825	valid_1's binary_logloss: 0.482279
[30]	training's binary_logloss: 0.377242	valid_1's binary_logloss: 0.456641
[40]	training's binary_logloss: 0.345424	valid_1's binary_logloss: 0.447083
[50]	training's binary_logloss: 0.323113	valid_1's binary_logloss: 0.440407
[60]	training's binary_logloss: 0.302727	valid_1's binary_logloss: 0.434527
[70]	training's binary_logloss: 0.285597	valid_1's binary_logloss: 0.434932
Early stopping, best iteration is:
[66]	training's binary_logloss: 0.293072	valid_1's binary_logloss: 0.433251


[I 2020-04-30 02:31:39,760] Finished trial#1 resulted in value: 0.4332512137886331. Current best value is 0.4332512137886331 with parameters: {'max_bin': 427, 'num_leaves': 79}.
/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning:

Using categorical_feature in Dataset.



...

[I 2020-04-30 02:32:03,166] Finished trial#37 resulted in value: 0.4332512137886331. Current best value is 0.4332512137886331 with parameters: {'max_bin': 427, 'num_leaves': 79}.
/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning:

Using categorical_feature in Dataset.



Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.505699	valid_1's binary_logloss: 0.532106
[20]	training's binary_logloss: 0.427825	valid_1's binary_logloss: 0.482279
[30]	training's binary_logloss: 0.377242	valid_1's binary_logloss: 0.456641
[40]	training's binary_logloss: 0.345424	valid_1's binary_logloss: 0.447083
[50]	training's binary_logloss: 0.323113	valid_1's binary_logloss: 0.440407
[60]	training's binary_logloss: 0.302727	valid_1's binary_logloss: 0.434527
[70]	training's binary_logloss: 0.285597	valid_1's binary_logloss: 0.434932
Early stopping, best iteration is:
[66]	training's binary_logloss: 0.293072	valid_1's binary_logloss: 0.433251


[I 2020-04-30 02:32:03,574] Finished trial#38 resulted in value: 0.4332512137886331. Current best value is 0.4332512137886331 with parameters: {'max_bin': 427, 'num_leaves': 79}.
/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning:

Using categorical_feature in Dataset.



Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.505699	valid_1's binary_logloss: 0.532106
[20]	training's binary_logloss: 0.427825	valid_1's binary_logloss: 0.482279
[30]	training's binary_logloss: 0.377242	valid_1's binary_logloss: 0.456641
[40]	training's binary_logloss: 0.345424	valid_1's binary_logloss: 0.447083
[50]	training's binary_logloss: 0.323113	valid_1's binary_logloss: 0.440407
[60]	training's binary_logloss: 0.302727	valid_1's binary_logloss: 0.434527
[70]	training's binary_logloss: 0.285597	valid_1's binary_logloss: 0.434932
Early stopping, best iteration is:
[66]	training's binary_logloss: 0.293072	valid_1's binary_logloss: 0.433251


[I 2020-04-30 02:32:03,992] Finished trial#39 resulted in value: 0.4332512137886331. Current best value is 0.4332512137886331 with parameters: {'max_bin': 427, 'num_leaves': 79}.


In [22]:
study.best_params

{'max_bin': 427, 'num_leaves': 79}

In [23]:
params = {
    'objective': 'binary',
    'max_bin': study.best_params['max_bin'],
    'learning_rate': 0.05,
    'num_leaves': study.best_params['num_leaves']
}

lgb_train = lgb.Dataset(X_train, y_train,
                                         categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train,
                                         categorical_feature=categorical_features)

model = lgb.train(params, lgb_train,
                               valid_sets=[lgb_train, lgb_eval],
                               verbose_eval=10,
                               num_boost_round=1000,
                               early_stopping_rounds=10)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning:

Using categorical_feature in Dataset.



Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.505699	valid_1's binary_logloss: 0.532106
[20]	training's binary_logloss: 0.427825	valid_1's binary_logloss: 0.482279
[30]	training's binary_logloss: 0.377242	valid_1's binary_logloss: 0.456641
[40]	training's binary_logloss: 0.345424	valid_1's binary_logloss: 0.447083
[50]	training's binary_logloss: 0.323113	valid_1's binary_logloss: 0.440407
[60]	training's binary_logloss: 0.302727	valid_1's binary_logloss: 0.434527
[70]	training's binary_logloss: 0.285597	valid_1's binary_logloss: 0.434932
Early stopping, best iteration is:
[66]	training's binary_logloss: 0.293072	valid_1's binary_logloss: 0.433251


In [25]:
y_pred = (y_pred > 0.5).astype(int)